# Single Allocation P-hub Location Median Problem using Hybrid Genetic Algorithm

## Importing Package and Dataset

In [1]:
# Importing Package and Libraries
import numpy as np
import pandas as pd
import random
import itertools
import time
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [2]:
# Importing Dataset Flow
# This dataset consists of flow from each nodes
CAB_10_nodes_flow = pd.read_csv("10_nodes_CAB_flow.csv", delimiter= ";", header = None)
CAB_25_nodes_flow = pd.read_csv("25_nodes_CAB_flow.csv", delimiter= ";", header = None)
TR_55_nodes_flow = pd.read_csv("55_nodes_TR_flow.csv", delimiter= ";", header = None)
TR_81_nodes_flow = pd.read_csv("81_nodes_TR_flow.csv", delimiter= ";", header = None)
RGP_100_nodes_flow = pd.read_csv("100_nodes_RGP_flow.csv", delimiter= ";", header = None)                                     

In [3]:
CAB_10_nodes_flow

,0,1,2,3,4,5,6,7,8,9
0,0,6469,7629,20036,4690,6194,11688,2243,8857,7248
1,6469,0,12999,13692,3322,5576,3878,3202,6699,4198
2,7629,12999,0,35135,5956,14121,5951,5768,16578,4242
3,20036,13692,35135,0,19094,35119,21423,27342,51341,15826
4,4690,3322,5956,19094,0,7284,3102,1562,7180,1917
5,6194,5576,14121,35119,7284,0,5023,3512,10419,3543
6,11688,3878,5951,21423,3102,5023,0,11557,6479,34261
7,2243,3202,5768,27342,1562,3512,11557,0,5615,7095
8,8857,6699,16578,51341,7180,10419,6479,5615,0,4448
9,7248,4198,4242,15826,1917,3543,34261,7095,4448,0


In [5]:
# Importing Dataset Cost
# This dataset consists of cost from each nodes
CAB_10_nodes_cost = pd.read_csv("10_nodes_CAB_cost.csv", delimiter= ";", header = None)
CAB_25_nodes_cost = pd.read_csv("25_nodes_CAB_cost.csv", delimiter= ";", header = None)
TR_55_nodes_cost = pd.read_csv("55_nodes_TR_cost.csv", delimiter= ";", header = None)
TR_81_nodes_cost = pd.read_csv("81_nodes_TR_cost.csv", delimiter= ";", header = None)
RGP_100_nodes_cost = pd.read_csv("100_nodes_RGP_cost.csv", delimiter= ";", header = None)

In [6]:
CAB_10_nodes_cost

,0,1,2,3,4,5,6,7,8,9
0,0.0000,576.9631,946.4954,597.5972,373.8127,559.7673,709.0215,1208.3280,603.6477,695.2080
1,576.9631,0.0000,369.5327,613.0386,429.1079,312.8831,1196.4890,1502.1400,405.8975,1241.9610
2,946.4954,369.5327,0.0000,858.3308,749.6018,556.0706,1541.2730,1764.7910,621.3306,1603.1650
3,597.5972,613.0386,858.3308,0.0000,255.0303,311.3071,790.1213,907.4331,237.0703,932.2173
4,373.8127,429.1079,749.6018,255.0303,0.0000,225.8954,794.1726,1080.3740,238.9440,879.5647
5,559.7673,312.8831,556.0706,311.3071,225.8954,0.0000,1009.6890,1216.8680,94.2588,1104.5740
6,709.0215,1196.4890,1541.2730,790.1213,794.1726,1009.6890,0.0000,663.8762,982.7378,221.4220
7,1208.3280,1502.1400,1764.7910,907.4331,1080.3740,1216.8680,663.8762,0.0000,1143.7910,874.5181
8,603.6477,405.8975,621.3306,237.0703,238.9440,94.2588,982.7378,1143.7910,0.0000,1094.9060
9,695.2080,1241.9610,1603.1650,932.2173,879.5647,1104.5740,221.4220,874.5181,1094.9060,0.0000


## Random Initial Solution and Population

In [7]:
# Random Initial Solution Function

def random_initial_solution (cost_matrix, number_hubs):
    
    number_nodes = cost_matrix.shape[0] # Number of Nodes is based on Cost Matrix Shape.
    nodes = range(1, number_nodes + 1) # Nodes is range from 1 to total nodes + 1
    
    hubs = random.sample(nodes, number_hubs) # Select the hubs randomly.
    spokes = [node for node in nodes if node not in hubs] # Spokes is node which is not in the hub list defined before.
    initial_solution = [0]*number_nodes # Initial Solution is array format with width total number of nodes. As an initial solution, the array consists of zero value.
    
    for node in nodes:
        if node in hubs: # If node is listed on hub list
            initial_solution [node - 1] = node # Then Initial Solution for Node - 1 equals with Node
        else:
            hub_spoke_cost = {hub : cost_matrix[node - 1][hub - 1] for hub in hubs}
            initial_solution[node - 1] = min(hub_spoke_cost, key = hub_spoke_cost.get)
    
    return initial_solution # Initial Solution for Hub Selection and Assignment

In [8]:
# Test Random Initial Function

test_solution_1 = random_initial_solution (CAB_10_nodes_cost, 3)

print ('Initial Solution Assigment: ', test_solution_1)
print ('Hub: ', set (test_solution_1))

Initial Solution Assigment:  [5, 9, 9, 9, 5, 9, 8, 8, 9, 8]
Hub:  {8, 9, 5}


In [9]:
# Initial Population based on Random Initial Solution

def initial_population (cost_matrix, n_hub, n_solution):
        # Generate Random Initial Solution and store into DataFrame
        df_random_solutions = {c: random_initial_solution (cost_matrix, n_hub)
                                for c in range(n_solution)}
    
        df_random_solutions = pd.DataFrame (df_random_solutions)

        initial_population = [tuple (list(df_random_solutions[i])) for i in range(0, n_solution)]
    
        in_population = [list(individual) for individual in initial_population]
        
        return in_population

In [10]:
# Test Population Function

initial_population_test1 = initial_population (CAB_25_nodes_cost, 3, 100)

## Cost Function

In [11]:
# Flow loc function
# Flow loc function is used to get economic of scale based on piece wise

def flow_loc_function (flow):
    if flow < 0:
        raise ValueError ('No negative value')
    elif flow < 50000:
        unit_flow_cost = 0 + 1*flow
    elif flow < 100000:
        unit_flow_cost = 10000 + 0.8*flow
    elif flow < 200000:
        unit_flow_cost = 30000 + 0.6*flow
    elif flow >= 200000:
        unit_flow_cost = 70000 + 0.4*flow

    return unit_flow_cost

In [12]:
# Test Flow Loc Function

test_flow_loc1 = flow_loc_function (10000)

print ('Unit Flow Cost: ', test_flow_loc1)

Unit Flow Cost:  10000


In [13]:
# Total Network cost function

def network_cost (solution, flow_matrix, cost_matrix):
    
    number_nodes = cost_matrix.shape[0]
    cost = 0 # Cost is initialized with zero value

    spoke_to_hub_cost = 0
    for node_1 in range(number_nodes): # For Node 1 in the range of number nodes
        for node_2 in range(number_nodes): # For Node 2 in the range of number nodes
            spoke_to_hub_cost += (flow_matrix[node_1][node_2] * (cost_matrix[node_1][solution[node_1]-1] + cost_matrix[solution[node_2]-1][node_2]))

    hub = set (solution) # Based on solution, check the unique number, which are the hubs number.

    hub_to_hub_cost = 0
    for hub_k in (hub): # Choose Hub K  
        for hub_m in (hub): # Choose Hub M
            if hub_k != hub_m: # Hub K not equals with Hub M
                hub_to_hub_flow = 0
                for node_1 in range (number_nodes): # For all Node 1 in range of number nodes
                    for node_2 in range (number_nodes): # For all Node 2 in range of number nodes
                        if node_1 != node_2:
                            if (solution[node_1] == hub_k) and (solution[node_2]) == hub_m:
                                hub_to_hub_flow += flow_matrix[node_1][node_2]

                hub_to_hub_cost += flow_loc_function (hub_to_hub_flow) * cost_matrix[hub_k-1][hub_m-1]
    
    cost = spoke_to_hub_cost + hub_to_hub_cost

    return cost

In [14]:
# Test Cost Network

test_cost1 = network_cost (test_solution_1, CAB_10_nodes_flow, CAB_10_nodes_cost)

print ('Total Cost Network: ', test_cost1)

Total Cost Network:  985879568.8086001


## Neighbourhood Structure (NS)

Neighbourhood structure is needed to be defined in the case of heuristic, including Genetic Algorithm. Neighboorhood Structure is used during Mutation Operator phase. There are several methods of mutation operator, such as Shift and Exchange Methods.

In [15]:
# Neighbourhood Structure 1: Relocate Hub - Swap Hub (Random Hub to Non Hub (Spoke), and Random Non Hub (Spoke) to Hub)
    # Paper 1: Solution Algorithm for the capacitated single allocation hub location problem - 1999
    # Paper 2: Efficient Algorithm for the uncapacitated single allocation p-hub median problem - 1996
    # Paper 3: Efficient Simulated Annealing based solution approaches to the competitive single and multiple allocation hub location problem

def NS1 (solution, cost_matrix):

    node = cost_matrix.shape[0]

    hub_to_spoke = random.choice (list(set(solution))) # Selecting randomly from solution list (hub list) to be a spoke.

    spokes = [node for node in range(1, node+1) if node not in set(solution)] # Spoke is in the range 1 to node+1 if it is not in solution list (hub list).

    spoke_to_hub = random.choice (spokes) # Selecting randomly spokes (non hub) from spoke list (spokes) to be a hub.   

    solution = [spoke_to_hub if x == hub_to_spoke else x for x in solution] # Solution is  new hub (from spoke) if x equals with new spoke (from hub), else it is x for x in solution.

    solution[spoke_to_hub-1] = spoke_to_hub # Solution (Spoke to hub - 1) equals with new hub (from spoke).

    return solution

In [16]:
# Neighbourhood Structure 1

solution_before_NS1 = [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]

solution_after_NS1 = NS1 (solution_before_NS1, CAB_25_nodes_cost)

print ('Solution Before NS1', solution_before_NS1 )
print ('Solution After NS1', solution_after_NS1 )

Solution Before NS1 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]
Solution After NS1 [23, 23, 3, 6, 23, 6, 6, 6, 6, 6, 6, 6, 3, 23, 6, 6, 23, 23, 3, 23, 23, 6, 23, 23, 23]


In [17]:
# Neighbourhood Structure 2: Swap Nodes: Swap the allocations of two (randomly chosen) non hub nodes from different cluster.
    # Paper 1: Solution Algorithm for the capacitated single allocation hub location problem - 1999
    # Paper 2: Mutation Procedure - Solving the uncapacitated hub location problem using genetic algorithm - 2005
    # Paper 3: Using simulated annealing to solve the p-hub median problem

def NS2 (solution, cost_matrix):

    node = cost_matrix.shape[0]

    hubs = list(set(solution)) # Hubs is based on list of solution.

    hubs_chosen = random.sample (hubs, 2) # Select randomly based on list of solution. 2 is number of items to be returned.

    hub_spoke  = {}
    for hub in set(solution):
        hub_spoke[hub] = [i+1 for i,val in enumerate(solution) if val==hub]
        for spokes in list(hub_spoke.values()):
            hub_spoke[hub] = [index for index in spokes if index != hub-1] #hubs and their spokes' indexes.

    l1 = hub_spoke[hubs_chosen[0]] # l1 is hub chosen first.
    l2 = hub_spoke[hubs_chosen[1]] # l2 is hub chosed second.
    
    spoke1 = random.choice(l1)
    spoke2 = random.choice(l2)

    solution[spoke1-1] = hubs_chosen[1]
    solution[spoke2-1] = hubs_chosen[0]
    
    solution[hubs_chosen[0]-1] = hubs_chosen[0]
    solution[hubs_chosen[1]-1] = hubs_chosen[1]
    
    return solution

In [18]:
# Test Neighbourhood Structure 2

solution_before_NS2 = [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]

print ('Solution before NS2', solution_before_NS2)

solution_after_NS2 = NS2 (solution_before_NS2, CAB_25_nodes_cost)

print ('Solution after NS2', solution_after_NS2)

Solution before NS2 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]
Solution after NS2 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 6, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]


In [19]:
# Neighbourhood Structure 3: Swapping hubs - Swaping the nodes of two hubs

def NS3 (solution, cost_matrix):

    node = cost_matrix.shape[0]

    hubs = list(set(solution))
    hubs_chosen = random.sample(hubs, 2)

    hub_spoke = {}
    for hub in set(solution):
        hub_spoke[hub] = [i for i,val in enumerate(solution) if val==hub]#it takes the idexes of the hubs in the solution
        for spokes in list(hub_spoke.values()):
            hub_spoke[hub] = [index for index in spokes if index != hub-1] #hubs and their spokes' indexes 
    
    for i in hub_spoke[hubs_chosen[0]]:
        solution[i] = hubs_chosen[1]
    for i in hub_spoke[hubs_chosen[1]]:
        solution[i] = hubs_chosen[0]
    
    return solution

In [20]:
# Test Neighbourhood Structure 3

solution_before_NS3 = [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]

print ('Solution before NS3', solution_before_NS3)

solution_after_NS3 = NS3 (solution_before_NS3, CAB_25_nodes_cost)

print ('Solution after NS3', solution_after_NS3)

Solution before NS3 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]
Solution after NS3 [3, 3, 3, 6, 3, 6, 6, 6, 6, 6, 6, 6, 21, 3, 6, 6, 3, 3, 21, 3, 21, 6, 6, 3, 3]


In [21]:
# Neighbourhood Structure 4: Reallocate a node - Change the allocation of a randomly chosen non hub node to a different randomly chosen hub
    # Paper 1: Solution Algorithm for the capacitated single allocation hub location problem - 1999
    # Paper 2: Efficient Algorithm for the uncapacitated single allocation p-hub median problem - 1996
    # Paper 3: Efficient Simulated Annealing based solution approaches to the competitive single and multiple allocation hub location problem
    # Paper 4: Mutation Procedure - Solving the uncapacitated hub location problem using genetic algorithm - 2005

def NS4 (solution, cost_matrix):

    node = cost_matrix.shape[0]

    hubs = list(set(solution))
    nodes = list(range(1,node+1))
    for hub in hubs:
        nodes.remove(hub)

    spoke = random.choice(nodes)

    hub1 = solution[spoke-1]
    hubs.remove(hub1)
    hub2 = random.choice(hubs)

    solution[spoke-1] = hub2
    
    return solution

In [22]:
solution_before_NS4 = [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]

print ('Solution before NS4', solution_before_NS4)

solution_after_NS4 = NS4 (solution_before_NS4, CAB_25_nodes_cost)

print ('Solution after NS4', solution_after_NS4)

Solution before NS4 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]
Solution after NS4 [21, 3, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]


In [23]:
# Neighbourhood Structure 5: Local Search using Reallocate Node

def NS5 (solution, flow_matrix, cost_matrix):
    
    # Initial Solution
    initial_solution = solution
    initial_solution_cost = network_cost (solution, flow_matrix, cost_matrix)
    hubs = list(set(initial_solution))
    
    # Best Solution
    best_solution = initial_solution.copy()
    best_cost = initial_solution_cost
    
    # Best Neighbourhood
    best_solution_neighbourhood = None
    best_cost_neighbourhood = float ('inf')

    for node in range (1, len (initial_solution)+1):
        if node in hubs:
            continue
        for new_hub in hubs:
            if new_hub == initial_solution[node-1]:
                continue
            else:
                neighbour_solution_result = initial_solution.copy()
                neighbour_solution_result[node-1] = new_hub
                neighbour_cost = network_cost (neighbour_solution_result, flow_matrix, cost_matrix)

            if neighbour_cost < best_cost_neighbourhood:
                best_cost_neighbourhood = neighbour_cost
                best_solution_neighbourhood = neighbour_solution_result.copy()
                if neighbour_cost < best_cost:
                    best_cost = neighbour_cost
                    best_solution = neighbour_solution_result.copy()
                else:
                    best_cost = best_cost
                    best_solution = best_solution
            else:
                best_cost_neighbourhood = best_cost_neighbourhood
                best_solution_neighbourhood = best_solution_neighbourhood

    return best_cost, best_solution

In [24]:
solution_before_NS5 = [4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18, 4, 4, 18, 18, 12, 18, 4, 4, 4, 18, 18]

print ('Solution before NS5', solution_before_NS5)

solution_after_NS5 = NS5 (solution_before_NS5, CAB_25_nodes_flow, CAB_25_nodes_cost)

print ('Solution after NS5', solution_after_NS5)

Solution before NS5 [4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18, 4, 4, 18, 18, 12, 18, 4, 4, 4, 18, 18]
Solution after NS5 (8591219259.912952, [4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18, 4, 4, 18, 18, 12, 18, 4, 12, 4, 18, 18])


## Genetic Algorithm

In Genetic Algorithm, there are several things to be considered, which are Fitness, Operator Selection Methods, Cross Over & Off Spring, and Mutation.

### Chromosome Fitness
Fitness of model can be fined by the cost

In [25]:
# Chromosome Fitness
def fitness_evaluation (population, flow_matrix, cost_matrix):
    
    solution_cost = [] # Creating an array
    
    for solution in population: # For all chromosome/solution in population
        cost = network_cost (solution, flow_matrix, cost_matrix)
        solution_cost.append((cost, solution))
 
    return solution_cost

### Operator Selection Methods
- There are several operator selection methods, such as Roulette Wheel Selection, Boltzman Selection, Tournament Selection, and Rank Selection
- Operator Selection is used to replace the lowest one with the best one.
- Tournament Selection will select the individuals with uniform probability with replacement.

In [26]:
# Tournament Selection Function

def tournament_selection (fitness_values_and_solutions, T_Individual): 

    chosen_individuals1 = random.sample (fitness_values_and_solutions, T_Individual) 
    # Random select solution which contains fintess value as defined on fintess evaluation function.
    # T is total individuals selected.

    parent1_fitness = min([individual[0] for individual in chosen_individuals1]) # Parent 1 Fitness is selected based on the minimum fitness (the best one or minimum cost)

    for individual in chosen_individuals1:
        if individual[0] == parent1_fitness:
            parent1 = individual

    chosen_individuals2 = random.sample(fitness_values_and_solutions, T_Individual)
    # Random select solution which contains fintess value as defined on fintess evaluation function.
    # T is total individuals selected
    
    parent2_fitness = min([individual[0] for individual in chosen_individuals2]) # Parent 2 Fitness is selected based on the minimum fitness (the best one or minimum cost)
    
    for individual in chosen_individuals2:
        if individual[0] == parent2_fitness:
            parent2 = individual
            
    parents = [parent[1] for parent in [parent1, parent2] ]
    
    return parents

### Cross Over and Offspring

- In order to create new generation, we need to cross over in cross over point. It can be single point cross over, two point cross over, multiple point cross over, or uniform cross over.
- There are two terminologies which are Parents and Off Spring. Parent is the initial solution and off spring is the new solution.
- Cross over will combine two parents, which will result into two off springs.
- Cross over wont guarantee a fitter solution.
- While doing cross over, we need to decide the cross over point.

In [27]:
# Cross-Over Function

def crossover (parents, cost_matrix):

    node = cost_matrix.shape[0] # Number of Nodes is based on Cost Matrix Shape.

    # Off spring
    offspring = [0]*node # Offspring zero array at first.

    # Random Point for Cross Over
    random_point = random.choice (range(0, node)) # Using randomization while choosing cross over point. The range is the total node (array).
    
    # Chosing parent 1
    parent1 = parents[0] # Selecting random parents as parent1 based on parents list as defined.

    for i in range (0, random_point+1): # For all number from index 0 to random point (Decided above) + 1 then
        offspring.pop(i) # remove values in the range 
        offspring.insert (i, parent1[i]) # then insert in the position i with parent1 with index i.
 
    # Chosing parent 2
    parent2 = parents[1] 
            
    for i in range (random_point, node): # For all number from index 0 to random point (Decided above) + 1 then
        offspring.pop (i) # remove values in the range
        offspring.insert (i, parent2[i]) # then insert in the position i with parent1 with index i.
    
    return offspring # Offspring is the new chromosome based on crossover two parents.

### Offspring Repair or Rearrangement

- Offspring Rearrangement is needed for Assign Array if the hub chosen is not valid anymore.
- Rearrangement new hub is based on distance matrix.

In [28]:
# Offspring Repair or Rearrangement Function

def offspring_repair (offspring, cost_matrix,  n_hub):
 
    node = cost_matrix.shape[0]

    hubs_offspring = []
    
    if len(set(offspring)) == n_hub:
        for hub in set(offspring):
            offspring[hub-1] = hub
    
    elif len(set(offspring)) > n_hub:  
        for i in range (1500) :
            if len(set(offspring)) == n_hub:
                break
            for hub in set(offspring):
                for i in range(0, node):
                    if offspring[i] not in hubs_offspring:
                        hubs_offspring.append(hub)                

            hub_spoke_indexes = {}

            for hub in hubs_offspring:
                hub_spoke_indexes[hub] = [i for i,val in enumerate(offspring) if val == hub] 
            hub_to_remove = hubs_offspring.pop(-1)
            
            hubs = list(set(hubs_offspring))
            chosen_hub = random.choice(hubs)
            
            for index in hub_spoke_indexes[hub_to_remove]:
                offspring.pop(index)
                offspring.insert(index, chosen_hub)
            for hub in hubs:
                offspring.pop(hub-1)
                offspring.insert(hub-1, hub)

    return offspring

In [29]:
# Feasibility Offspring

def feasibility_offspring (offspring, n_hubs):
    
    if len(set(offspring)) == n_hubs: # and offspring != list(parents[0]) and offspring != list(parents[1]):
        return 1
    else:
        return 0

### Mutation

- Mutation is executed for offspring. 
- Mutation will replace one of value in solution with another value. For example: Initial : 001001, Replace second with "1", then 011001 or using neighbourhood structure.
- There are two types mutations example: Shift and Exchange.
    - Shift is to select a spoke and reassign to new hub randomly. If there is only one hub in the string, then this method is not applicable.
    - Exchange  is  to select two spokes and switch the hub or assignments. If there is only one hub or one spoke, then this method is not applicable.

In [30]:
# Mutation Function

def mutation (offspring, cost_matrix, NS_type):

    # operation_list = ["NS4", "NS3", "NS2", "NS1"] # Neighbourhood structures have been defined before. In this case, we choose NS4, NS3, and NS1 as neighbourhood structure.

    mutation_procedure = NS_type # Operation method are chosen randomly based on operation list.

    if mutation_procedure == "NS4":
        return NS4 (offspring, cost_matrix) # If operation method select NS4 then execute NS4 function.

    elif mutation_procedure == "NS3":
        return NS4 (offspring, cost_matrix) # If operation method select NS3 then execute NS3 function. 
    
    elif mutation_procedure == "NS2":
        return NS2 (offspring, cost_matrix) # If operation method select NS3 then execute NS2 function.
    
    else:
        return NS1 (offspring, cost_matrix) # If operation method select NS1 then execute NS1 function.

# Hybrid Genetic Algorithm with Tabu Search

In [31]:
# Hybrid Genetic Algorithm and Tabu Search Function

def hybrid_GA_TS (n_hub, T_Individual, flow_matrix, cost_matrix, n_solution, iter, iter_tabu, NS_List, tabu_capacity):
    
    # Genetic Algorithm Process

    start = time.time()
    offsprings = []
    
    # The initial population using least cost initial population defined above
    init_pop = initial_population (cost_matrix, n_hub, n_solution)
    
    # Population Evaluation using fitness evaluation function defined above
    pop_evaluation = fitness_evaluation (init_pop, flow_matrix, cost_matrix)

    best_cost = float ('inf')
    best_solution = None
    counter = 0
    
    while counter < iter:
        # Tournament Selection
        parents = tournament_selection (pop_evaluation, T_Individual) # Choose parent as operator selection using Tournament Selection
        
        total_offsprings = 0
        while total_offsprings < n_solution:
            
            # Cross Over, Offspring Repairement and New Fitness Evaluation
            offspring_crossovered = crossover (parents, cost_matrix)
            offspring_crossovered_fixed = offspring_repair (offspring_crossovered, cost_matrix, n_hub)
            evaluation_c = network_cost (offspring_crossovered_fixed, flow_matrix, cost_matrix)
                
            # Mutation Process - Using two type of mutation process NS1 and NS2
            # Source: Solving the uncapacitated hub location problem using genetic algorithms

            operator_selected = random.choice(NS_List)
            offspring_crossovered_mutated_final = mutation (offspring_crossovered_fixed, cost_matrix, operator_selected) 
            evaluation_cm_final = network_cost (offspring_crossovered_mutated_final, flow_matrix, cost_matrix)

            if evaluation_cm_final < evaluation_c: # If Fitness Evaluation of Mutation is below (better) than before than mutation (only cross-over), than accept
                final_offspring = offspring_crossovered_mutated_final
                final_offspring_cost = evaluation_cm_final
                offsprings.append ((final_offspring_cost, final_offspring))
            else:
                final_offspring = offspring_crossovered_fixed
                final_offspring_cost = evaluation_c
                offsprings.append ((final_offspring_cost, final_offspring))
            if final_offspring_cost < best_cost:
                best_solution = final_offspring.copy()
                best_cost = final_offspring_cost
            else:
                best_solution = best_solution
                best_cost = best_cost
            total_offsprings +=1

        # Combine with Initial Population Evaluation + Offspring, sorted, and take n solution
        initpopeval_and_offsprings = offsprings + pop_evaluation
        initpopeval_and_offsprings.sort()
        initpopeval_and_offsprings = initpopeval_and_offsprings[0:n_solution]

        # Pop Evaluation for next iteration
        pop_evaluation = initpopeval_and_offsprings # Initial Population = Offsprings
        offsprings = []
        
        counter +=1 # Add counter for iteration

    best_solution_genetic_algorithm = pop_evaluation[:5]
            
    # Tabu Search process
    # Best Overall
    best_solution = best_solution
    best_cost = best_cost

    # Best Solution Neigbourhood for certain iteration
    best_solution_neighbourhood = None
    best_cost_neighbourhood = float ('inf')

    # Tabu Setting
    become_tabu = None
    tabu_list = []
    tabu_capacity = tabu_capacity
    tabu_counter = 0

    # Do Tabu Search for all chromosome in Genetic Algorithm
    for i in range (len (best_solution_genetic_algorithm)):

        # Choose initial solution based on chromosome
        initial_solution = best_solution_genetic_algorithm[i][1]
        initial_solution_cost = best_solution_genetic_algorithm [i][0]
        hubs = list(set(initial_solution))
        if initial_solution_cost < best_cost:
            best_cost = initial_solution_cost
            best_solution = initial_solution.copy()
        else:
            best_cost = best_cost
            best_solution = best_solution
        
        # Do Tabu search for several iteration
        while tabu_counter < iter_tabu:

            # Reset Neighbourhood
            best_solution_neighbourhood = None
            best_cost_neighbourhood = float ('inf')
            
            for node in range (1, len (initial_solution)+1):
                if node in hubs:
                    continue
                if node in tabu_list:
                    continue
                for new_hub in hubs:
                    if new_hub == initial_solution[node-1]:
                        continue
                    else:
                        neighbour_solution_result = initial_solution.copy()
                        neighbour_solution_result[node-1] = new_hub
                        
                        neighbour_cost = network_cost (neighbour_solution_result, flow_matrix, cost_matrix)

                        if neighbour_cost < best_cost_neighbourhood:
                            best_cost_neighbourhood = neighbour_cost
                            best_solution_neighbourhood = neighbour_solution_result.copy()
                            become_tabu = [] # Delete first
                            become_tabu = node # Replace New One
                            if neighbour_cost < best_cost: # Compare to all global best solution
                                best_cost = neighbour_cost
                                best_solution = neighbour_solution_result.copy()
                            else:
                                best_cost = best_cost
                                best_solution = best_solution
                        
                        else:
                            best_cost_neighbourhood = best_cost_neighbourhood
                            best_solution_neighbourhood = best_solution_neighbourhood


            if len (tabu_list) >= tabu_capacity:
                tabu_list.pop(0)
                tabu_list.append(become_tabu)
            else:
                tabu_list.append(become_tabu)
                    
            initial_solution = best_solution_neighbourhood.copy()
            hubs = list(set(initial_solution))

            tabu_counter +=1

        end = time.time()

        total_time = end - start

    return best_cost, best_solution, total_time, total_time/iter

## Simulate Hybrid Genetic Algorithm and Tabu Search

In [32]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with CAB 10 Dataset - 3 Hubs

total_running = 10
minimum_value_cab10_3_ga_ts = []

for i in range(total_running):
    random.seed(i+100)
    result = hybrid_GA_TS (3, 10, CAB_10_nodes_flow, CAB_10_nodes_cost, 100, 50, 10, ['NS1', 'NS2'], 5)
    minimum_value_cab10_3_ga_ts.append(result)

minimum_value_dataframe_cab10_3_ga_ts = pd.DataFrame (minimum_value_cab10_3_ga_ts, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab10_3_ga_ts

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",8.134087,0.162682
1,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",8.072895,0.161458
2,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",8.050990,0.161020
3,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",8.071585,0.161432
4,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",8.064650,0.161293
5,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",8.083632,0.161673
6,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",8.074451,0.161489
7,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",8.084414,0.161688
8,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",8.064233,0.161285
9,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",8.097183,0.161944


In [44]:
minimum_value_dataframe_cab10_3_ga_ts.to_csv ('minimum_value_dataframe_cab10_3_ga_ts_50iter.csv')

In [45]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with CAB 25 Dataset - 3 Hubs

total_running = 10
minimum_value_cab25_3_ga_ts = []

for i in range(total_running):
    random.seed(i+100)
    result = hybrid_GA_TS (3, 10, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50, 10, ['NS1', 'NS2'], 10)
    minimum_value_cab25_3_ga_ts.append(result)

minimum_value_dataframe_cab25_3_ga_ts = pd.DataFrame (minimum_value_cab25_3_ga_ts, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_3_ga_ts

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.547169,1.030943
1,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",50.403022,1.008060
2,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.171748,1.023435
3,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",52.548409,1.050968
4,8.446904e+09,"[21, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 2...",52.187852,1.043757
5,8.446904e+09,"[21, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 2...",51.610033,1.032201
6,8.446904e+09,"[21, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 2...",51.953133,1.039063
7,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.871487,1.037430
8,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.435739,1.028715
9,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.948031,1.038961


In [46]:
minimum_value_dataframe_cab25_3_ga_ts.to_csv ('minimum_value_dataframe_cab25_3_ga_ts_50iter.csv')

In [49]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with CAB 25 Dataset - 5 Hubs

total_running = 10
minimum_value_cab25_5_ga_ts = []

for i in range(total_running):
    random.seed(i+100)
    result = hybrid_GA_TS (5, 10, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50, 10, ['NS1', 'NS2'], 10)
    minimum_value_cab25_5_ga_ts.append(result)

minimum_value_dataframe_cab25_5_ga_ts = pd.DataFrame (minimum_value_cab25_5_ga_ts, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_5_ga_ts

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,7.920597e+09,"[4, 17, 17, 4, 5, 4, 4, 4, 4, 4, 4, 12, 4, 14,...",59.152860,1.183057
1,7.829156e+09,"[1, 17, 17, 4, 4, 4, 4, 4, 4, 1, 4, 12, 1, 1, ...",58.655927,1.173119
2,7.822886e+09,"[1, 17, 17, 4, 4, 6, 4, 4, 6, 1, 4, 12, 1, 1, ...",58.543405,1.170868
3,7.817837e+09,"[1, 17, 17, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 14,...",58.196845,1.163937
4,7.773952e+09,"[1, 17, 17, 4, 4, 4, 4, 8, 4, 1, 4, 12, 1, 1, ...",58.704322,1.174086
5,7.678738e+09,"[1, 17, 17, 4, 4, 4, 7, 4, 4, 7, 4, 12, 1, 1, ...",59.639722,1.192794
6,7.773952e+09,"[1, 17, 17, 4, 4, 4, 4, 8, 4, 1, 4, 12, 1, 1, ...",59.874686,1.197494
7,7.675539e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",59.640609,1.192812
8,7.816650e+09,"[21, 17, 17, 21, 6, 6, 21, 21, 6, 21, 21, 12, ...",59.782368,1.195647
9,7.817837e+09,"[1, 17, 17, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 14,...",58.651938,1.173039


In [51]:
minimum_value_dataframe_cab25_5_ga_ts.to_csv ('minimum_value_dataframe_cab25_5_ga_ts_50iter.csv')

In [55]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with TR 55 Dataset - 3 Hubs

total_running = 10
minimum_value_tr55_3_ga_ts = []

for i in range(total_running):
    random.seed(i+100)
    result = hybrid_GA_TS (3, 10, TR_55_nodes_flow, TR_55_nodes_cost, 100, 10, 10, ['NS1', 'NS2'], 15)
    minimum_value_tr55_3_ga_ts.append(result)

minimum_value_dataframe_tr55_3_ga_ts = pd.DataFrame (minimum_value_tr55_3_ga_ts, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_tr55_3_ga_ts

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,2.814762e+10,"[34, 34, 34, 4, 4, 34, 30, 30, 34, 34, 4, 30, ...",73.076936,7.307694
1,2.829849e+10,"[19, 19, 19, 4, 4, 19, 30, 30, 19, 19, 4, 30, ...",77.549087,7.754909
2,2.946977e+10,"[45, 17, 17, 45, 45, 17, 30, 30, 17, 17, 45, 3...",72.954976,7.295498
3,2.911092e+10,"[45, 42, 42, 45, 45, 42, 30, 30, 42, 42, 45, 3...",76.906292,7.690629
4,2.909153e+10,"[45, 15, 15, 45, 45, 15, 30, 30, 15, 15, 45, 3...",75.465172,7.546517
5,2.991043e+10,"[45, 34, 34, 45, 45, 34, 12, 12, 34, 34, 12, 1...",74.670049,7.467005
6,2.963839e+10,"[1, 1, 45, 45, 45, 45, 30, 30, 1, 1, 45, 30, 3...",73.765996,7.376600
7,3.003738e+10,"[55, 55, 55, 4, 18, 4, 18, 18, 55, 55, 18, 18,...",73.993074,7.399307
8,3.029897e+10,"[28, 28, 28, 4, 4, 28, 30, 30, 28, 28, 4, 30, ...",71.747655,7.174766
9,2.830434e+10,"[55, 55, 55, 4, 4, 4, 30, 30, 55, 55, 4, 30, 3...",74.176867,7.417687


In [57]:
minimum_value_dataframe_tr55_3_ga_ts.to_csv ('minimum_value_dataframe_tr55_3_ga_ts_10iter.csv')

In [59]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with TR 55 Dataset - 5 Hubs

total_running = 10
minimum_value_tr55_5_ga_ts = []

for i in range(total_running):
    random.seed(i+100)
    result = hybrid_GA_TS (5, 10, TR_55_nodes_flow, TR_55_nodes_cost, 100, 10, 10, ['NS1', 'NS2'], 15)
    minimum_value_tr55_5_ga_ts.append(result)

minimum_value_dataframe_tr55_5_ga_ts = pd.DataFrame (minimum_value_tr55_5_ga_ts, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_tr55_5_ga_ts

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,2.677055e+10,"[19, 19, 17, 18, 18, 17, 18, 18, 17, 17, 18, 1...",106.379785,10.637979
1,2.566451e+10,"[55, 55, 17, 4, 4, 17, 30, 30, 17, 17, 4, 30, ...",110.740977,11.074098
2,2.569614e+10,"[34, 34, 34, 4, 31, 34, 8, 8, 34, 34, 31, 8, 2...",105.903820,10.590382
3,2.714313e+10,"[34, 34, 34, 18, 24, 34, 27, 18, 34, 34, 24, 1...",98.717450,9.871745
4,2.390635e+10,"[1, 1, 15, 4, 33, 15, 33, 4, 15, 15, 4, 26, 26...",103.582487,10.358249
5,2.684135e+10,"[1, 42, 42, 29, 31, 42, 12, 12, 42, 42, 31, 12...",112.454643,11.245464
6,2.527006e+10,"[19, 19, 10, 4, 33, 10, 33, 26, 10, 10, 4, 26,...",115.128082,11.512808
7,2.663803e+10,"[25, 15, 15, 4, 4, 15, 13, 26, 15, 15, 4, 26, ...",115.494072,11.549407
8,2.567681e+10,"[19, 19, 19, 4, 33, 19, 12, 12, 19, 19, 4, 12,...",105.030651,10.503065
9,2.756887e+10,"[54, 54, 54, 29, 32, 29, 33, 32, 54, 54, 32, 3...",112.609510,11.260951


In [60]:
minimum_value_dataframe_tr55_5_ga_ts.to_csv ('minimum_value_dataframe_tr55_5_ga_ts_10iter.csv')

In [64]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with TR 81 Dataset - 5 Hubs

total_running = 10
minimum_value_tr81_5_ga_ts = []

for i in range(total_running):
    random.seed(i+100)
    result = hybrid_GA_TS (5, 10, TR_81_nodes_flow, TR_81_nodes_cost, 100, 10, 15, ['NS1', 'NS2'], 30)
    minimum_value_tr81_5_ga_ts.append(result)

minimum_value_dataframe_tr81_5_ga_ts = pd.DataFrame (minimum_value_tr81_5_ga_ts, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_tr81_5_ga_ts

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,4.407718e+10,"[46, 46, 3, 60, 60, 6, 3, 60, 3, 3, 3, 46, 46,...",362.174945,36.217495
1,4.541351e+10,"[46, 46, 54, 60, 60, 68, 68, 60, 45, 45, 54, 4...",397.077319,39.707732
2,4.786432e+10,"[27, 27, 6, 27, 6, 6, 6, 27, 35, 16, 54, 27, 2...",423.265737,42.326574
3,4.814136e+10,"[70, 2, 16, 2, 71, 71, 70, 2, 16, 16, 16, 2, 2...",361.514274,36.151427
4,4.729738e+10,"[6, 23, 64, 25, 6, 6, 64, 25, 64, 64, 54, 23, ...",379.776480,37.977648
5,4.522969e+10,"[46, 46, 64, 69, 66, 66, 64, 69, 64, 64, 64, 6...",347.609500,34.760950
6,4.685962e+10,"[80, 80, 20, 58, 18, 18, 20, 58, 20, 54, 54, 5...",404.885249,40.488525
7,4.802847e+10,"[6, 23, 6, 23, 66, 6, 6, 23, 16, 16, 16, 23, 2...",381.369620,38.136962
8,4.720300e+10,"[6, 23, 43, 23, 66, 6, 43, 23, 43, 43, 43, 23,...",397.759793,39.775979
9,4.742460e+10,"[50, 23, 20, 23, 40, 40, 20, 23, 20, 41, 41, 2...",388.581705,38.858171


In [65]:
minimum_value_dataframe_tr81_5_ga_ts.to_csv ('minimum_value_dataframe_tr81_5_ga_ts_10iter.csv')

In [66]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with TR 81 Dataset - 7 Hubs

total_running = 10
minimum_value_tr81_7_ga_ts = []

for i in range(total_running):
    random.seed(i+100)
    result = hybrid_GA_TS (7, 10, TR_81_nodes_flow, TR_81_nodes_cost, 100, 10, 15, ['NS1', 'NS2'], 30)
    minimum_value_tr81_7_ga_ts.append(result)

minimum_value_dataframe_tr81_7_ga_ts = pd.DataFrame (minimum_value_tr81_7_ga_ts, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_tr81_7_ga_ts

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,4.150064e+10,"[1, 44, 11, 69, 6, 6, 11, 69, 35, 35, 11, 44, ...",605.882431,60.588243
1,4.575266e+10,"[68, 27, 15, 60, 60, 68, 15, 55, 15, 41, 41, 2...",625.544137,62.554414
2,4.355249e+10,"[33, 23, 71, 24, 71, 71, 35, 61, 35, 35, 34, 2...",527.949154,52.794915
3,4.477419e+10,"[38, 23, 42, 23, 6, 6, 42, 23, 9, 9, 6, 23, 23...",639.884589,63.988459
4,4.167491e+10,"[80, 80, 3, 21, 55, 3, 3, 55, 45, 45, 41, 21, ...",515.321665,51.532167
5,4.535693e+10,"[46, 46, 41, 28, 38, 78, 45, 28, 45, 45, 41, 4...",635.610249,63.561025
6,4.326655e+10,"[80, 80, 42, 28, 14, 14, 42, 28, 35, 35, 41, 4...",571.131677,57.113168
7,4.504924e+10,"[51, 27, 64, 28, 28, 81, 64, 28, 64, 64, 54, 4...",551.209266,55.120927
8,4.458604e+10,"[80, 80, 3, 61, 71, 71, 15, 61, 20, 54, 54, 80...",629.197101,62.919710
9,4.395565e+10,"[46, 46, 32, 13, 66, 66, 32, 12, 45, 45, 34, 1...",599.925745,59.992574


In [68]:
minimum_value_dataframe_tr81_7_ga_ts.to_csv ('minimum_value_dataframe_tr81_7_ga_ts_10iter.csv')

In [71]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with RGP 100 Dataset - 7 Hubs

total_running = 10
minimum_value_rgp100_7_ga_ts = []

for i in range(total_running):
    random.seed(i+100)
    result = hybrid_GA_TS (7, 10, RGP_100_nodes_flow, RGP_100_nodes_cost, 100, 10, 15, ['NS1', 'NS2'], 50)
    minimum_value_rgp100_7_ga_ts.append(result)

minimum_value_dataframe_rgp100_7_ga_ts = pd.DataFrame (minimum_value_rgp100_7_ga_ts, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_rgp100_7_ga_ts

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,1.398249e+11,"[28, 34, 52, 52, 51, 34, 33, 51, 33, 43, 34, 3...",1206.655002,120.665500
1,1.396672e+11,"[37, 90, 8, 52, 15, 8, 18, 8, 15, 15, 90, 15, ...",1277.940457,127.794046
2,1.392890e+11,"[5, 48, 49, 4, 5, 49, 64, 64, 64, 5, 4, 4, 4, ...",1146.200202,114.620020
3,1.401242e+11,"[58, 84, 9, 58, 74, 49, 58, 75, 9, 74, 75, 49,...",920.982014,92.098201
4,1.393528e+11,"[5, 36, 36, 36, 5, 5, 5, 64, 64, 10, 11, 11, 3...",995.228558,99.522856
5,1.388032e+11,"[5, 5, 88, 4, 5, 49, 4, 49, 49, 47, 27, 4, 27,...",1041.068660,104.106866
6,1.380402e+11,"[1, 54, 1, 20, 27, 20, 76, 54, 20, 1, 11, 11, ...",1034.959411,103.495941
7,1.406863e+11,"[48, 2, 21, 21, 48, 21, 99, 21, 17, 54, 2, 99,...",1165.539885,116.553989
8,1.393481e+11,"[73, 24, 15, 52, 15, 44, 73, 24, 44, 15, 24, 9...",1212.512166,121.251217
9,1.393651e+11,"[76, 5, 26, 29, 5, 34, 76, 26, 26, 71, 34, 34,...",922.317652,92.231765


In [72]:
minimum_value_dataframe_rgp100_7_ga_ts.to_csv ('minimum_value_dataframe_rgp100_7_ga_ts_10iter.csv')

In [73]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with RGP 100 Dataset - 10 Hubs

total_running = 10
minimum_value_rgp100_10_ga_ts = []

for i in range(total_running):
    random.seed(i+100)
    result = hybrid_GA_TS (10, 10, RGP_100_nodes_flow, RGP_100_nodes_cost, 100, 10, 15, ['NS1', 'NS2'], 50)
    minimum_value_rgp100_10_ga_ts.append(result)

minimum_value_dataframe_rgp100_10_ga_ts = pd.DataFrame (minimum_value_rgp100_10_ga_ts, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_rgp100_10_ga_ts

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,1.386032e+11,"[28, 24, 52, 24, 51, 43, 33, 24, 33, 43, 27, 3...",1928.243295,192.824329
1,1.378938e+11,"[37, 36, 36, 36, 15, 8, 18, 8, 15, 12, 36, 12,...",1848.106804,184.810680
2,1.387493e+11,"[28, 87, 15, 4, 15, 20, 87, 90, 20, 83, 90, 4,...",1941.401288,194.140129
3,1.379332e+11,"[72, 2, 8, 4, 5, 8, 87, 8, 96, 87, 41, 4, 94, ...",2026.534349,202.653435
4,1.387120e+11,"[73, 94, 26, 20, 64, 20, 73, 97, 20, 12, 41, 1...",2053.460346,205.346035
5,1.384078e+11,"[60, 86, 15, 20, 60, 20, 33, 51, 20, 26, 86, 3...",1539.453864,153.945386
6,1.374983e+11,"[28, 17, 25, 20, 27, 20, 73, 25, 20, 41, 27, 2...",1878.666656,187.866666
7,1.396871e+11,"[92, 7, 9, 20, 64, 20, 7, 64, 9, 9, 82, 64, 7,...",1954.170877,195.417088
8,1.380477e+11,"[47, 17, 9, 4, 90, 97, 33, 90, 9, 9, 90, 4, 94...",1933.361188,193.336119
9,1.378740e+11,"[79, 79, 8, 20, 64, 20, 33, 8, 9, 39, 79, 79, ...",1626.048787,162.604879


In [75]:
minimum_value_dataframe_rgp100_10_ga_ts.to_csv ('minimum_value_dataframe_rgp100_10_ga_ts_10iter.csv')

# Sensitivity Analysis

In [79]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with CAB 25 Dataset - 3 Hubs - Tabu Capacity = 5

total_running = 10
minimum_value_cab25_3_ga_ts_tab_cap5 = []

for i in range(total_running):
    random.seed()
    result = hybrid_GA_TS (3, 10, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50, 10, ['NS1', 'NS2'], 5)
    minimum_value_cab25_3_ga_ts_tab_cap5.append(result)

minimum_value_dataframe_cab25_3_ga_ts_tab_cap5 = pd.DataFrame (minimum_value_cab25_3_ga_ts_tab_cap5, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_3_ga_ts_tab_cap5

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.344071,1.026881
1,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.527212,1.030544
2,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",50.649658,1.012993
3,8.411089e+09,"[2, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 12, 4, 2, 4,...",51.059276,1.021186
4,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",50.816241,1.016325
5,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",50.876644,1.017533
6,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.071329,1.021427
7,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",50.733667,1.014673
8,8.411089e+09,"[2, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 12, 4, 2, 4,...",50.932936,1.018659
9,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",50.226697,1.004534


In [85]:
minimum_value_dataframe_cab25_3_ga_ts_tab_cap5.to_csv ('minimum_value_dataframe_cab25_3_ga_ts_tab_cap5.csv')

In [80]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with CAB 25 Dataset - 3 Hubs - Tabu Capacity = 10

total_running = 10
minimum_value_cab25_3_ga_ts_tab_cap10 = []

for i in range(total_running):
    random.seed()
    result = hybrid_GA_TS (3, 10, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50, 10, ['NS1', 'NS2'], 10)
    minimum_value_cab25_3_ga_ts_tab_cap10.append(result)

minimum_value_dataframe_cab25_3_ga_ts_tab_cap10 = pd.DataFrame (minimum_value_cab25_3_ga_ts_tab_cap10, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_3_ga_ts_tab_cap10

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.792366,1.035847
1,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.237303,1.024746
2,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.404343,1.028087
3,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.582076,1.031642
4,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",50.803819,1.016076
5,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.496834,1.029937
6,8.913559e+09,"[13, 20, 20, 20, 20, 20, 13, 13, 20, 13, 13, 1...",51.410623,1.028212
7,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.548072,1.030961
8,8.411089e+09,"[2, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 12, 4, 2, 4,...",51.401731,1.028035
9,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.607644,1.032153


In [86]:
minimum_value_dataframe_cab25_3_ga_ts_tab_cap10.to_csv ('minimum_value_dataframe_cab25_3_ga_ts_tab_cap10.csv')

In [81]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with CAB 25 Dataset - 3 Hubs - Tabu Capacity = 15

total_running = 10
minimum_value_cab25_3_ga_ts_tab_cap15 = []

for i in range(total_running):
    random.seed()
    result = hybrid_GA_TS (3, 10, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50, 10, ['NS1', 'NS2'], 15)
    minimum_value_cab25_3_ga_ts_tab_cap15.append(result)

minimum_value_dataframe_cab25_3_ga_ts_tab_cap15 = pd.DataFrame (minimum_value_cab25_3_ga_ts_tab_cap15, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_3_ga_ts_tab_cap15

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.140526,1.022811
1,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 12,...",51.397794,1.027956
2,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.241219,1.024824
3,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",50.014604,1.000292
4,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.816495,1.036330
5,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.090173,1.021803
6,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",51.629621,1.032592
7,8.446904e+09,"[21, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 2...",51.919637,1.038393
8,8.411089e+09,"[2, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 12, 4, 2, 4,...",51.770625,1.035413
9,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",50.946768,1.018935


In [87]:
minimum_value_dataframe_cab25_3_ga_ts_tab_cap15.to_csv ('minimum_value_dataframe_cab25_3_ga_ts_tab_cap15.csv')

In [88]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with CAB 25 Dataset - 5 Hubs - Tabu Capacity = 5

total_running = 10
minimum_value_cab25_5_ga_ts_tab_cap5 = []

for i in range(total_running):
    random.seed()
    result = hybrid_GA_TS (5, 10, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50, 10, ['NS1', 'NS2'], 5)
    minimum_value_cab25_5_ga_ts_tab_cap5.append(result)

minimum_value_dataframe_cab25_5_ga_ts_tab_cap5 = pd.DataFrame (minimum_value_cab25_5_ga_ts_tab_cap5, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_5_ga_ts_tab_cap5

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,7.822886e+09,"[1, 17, 17, 4, 4, 6, 4, 4, 6, 1, 4, 12, 1, 1, ...",61.114949,1.222299
1,7.658571e+09,"[4, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 14,...",60.226218,1.204524
2,7.675539e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",60.501439,1.210029
3,7.822886e+09,"[1, 17, 17, 4, 4, 6, 4, 4, 6, 1, 4, 12, 1, 1, ...",61.228176,1.224564
4,7.909196e+09,"[13, 17, 17, 4, 4, 4, 13, 8, 4, 13, 4, 12, 13,...",60.338739,1.206775
5,7.678738e+09,"[1, 17, 17, 4, 4, 4, 7, 4, 4, 7, 4, 12, 1, 1, ...",60.156557,1.203131
6,7.658571e+09,"[4, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 14,...",60.294184,1.205884
7,7.678738e+09,"[1, 17, 17, 4, 4, 4, 7, 4, 4, 7, 4, 12, 1, 1, ...",60.210881,1.204218
8,7.773952e+09,"[1, 17, 17, 4, 4, 4, 4, 8, 4, 1, 4, 12, 1, 1, ...",60.341377,1.206828
9,7.675539e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",60.377446,1.207549


In [89]:
minimum_value_dataframe_cab25_5_ga_ts_tab_cap5.to_csv ('minimum_value_dataframe_cab25_5_ga_ts_tab_cap5.csv')

In [83]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with CAB 25 Dataset - 5 Hubs - Tabu Capacity = 10

total_running = 10
minimum_value_cab25_5_ga_ts_tab_cap10 = []

for i in range(total_running):
    random.seed()
    result = hybrid_GA_TS (5, 10, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50, 10, ['NS1', 'NS2'], 10)
    minimum_value_cab25_5_ga_ts_tab_cap10.append(result)

minimum_value_dataframe_cab25_5_ga_ts_tab_cap10 = pd.DataFrame (minimum_value_cab25_5_ga_ts_tab_cap10, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_5_ga_ts_tab_cap10

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,7.658571e+09,"[4, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 14,...",61.063544,1.221271
1,7.912773e+09,"[4, 17, 17, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 14,...",59.296449,1.185929
2,7.822886e+09,"[1, 17, 17, 4, 4, 6, 4, 4, 6, 1, 4, 12, 1, 1, ...",60.557127,1.211143
3,7.675539e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",60.863253,1.217265
4,7.817837e+09,"[1, 17, 17, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 14,...",59.890236,1.197805
5,7.658571e+09,"[4, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 14,...",60.777907,1.215558
6,7.678738e+09,"[1, 17, 17, 4, 4, 4, 7, 4, 4, 7, 4, 12, 1, 1, ...",61.022015,1.220440
7,7.884077e+09,"[1, 17, 17, 4, 4, 4, 4, 4, 4, 1, 11, 12, 1, 1,...",61.002918,1.220058
8,8.168481e+09,"[4, 18, 18, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 18,...",59.525775,1.190516
9,7.773952e+09,"[1, 17, 17, 4, 4, 4, 4, 8, 4, 1, 4, 12, 1, 1, ...",59.659113,1.193182


In [90]:
minimum_value_dataframe_cab25_5_ga_ts_tab_cap10.to_csv ('minimum_value_dataframe_cab25_5_ga_ts_tab_cap10.csv')

In [84]:
# Simulate Hybrid Genetic Algorithm and Tabu Search with CAB 25 Dataset - 5 Hubs - Tabu Capacity = 15

total_running = 10
minimum_value_cab25_5_ga_ts_tab_cap15 = []

for i in range(total_running):
    random.seed()
    result = hybrid_GA_TS (5, 10, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50, 10, ['NS1', 'NS2'], 15)
    minimum_value_cab25_5_ga_ts_tab_cap15.append(result)

minimum_value_dataframe_cab25_5_ga_ts_tab_cap15 = pd.DataFrame (minimum_value_cab25_5_ga_ts_tab_cap15, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_5_ga_ts_tab_cap15

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,7.658571e+09,"[4, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 14,...",59.761920,1.195238
1,8.013842e+09,"[21, 18, 18, 21, 21, 18, 21, 21, 18, 21, 21, 1...",58.605406,1.172108
2,7.688855e+09,"[4, 18, 18, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 14,...",60.747386,1.214948
3,7.658571e+09,"[4, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 14,...",61.373111,1.227462
4,7.675539e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",61.128301,1.222566
5,7.675539e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",60.634837,1.212697
6,7.822886e+09,"[1, 17, 17, 4, 4, 6, 4, 4, 6, 1, 4, 12, 1, 1, ...",61.051326,1.221027
7,7.675539e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",60.279930,1.205599
8,7.658571e+09,"[4, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 14,...",60.408378,1.208168
9,7.678738e+09,"[1, 17, 17, 4, 4, 4, 7, 4, 4, 7, 4, 12, 1, 12,...",60.651480,1.213030


In [91]:
minimum_value_dataframe_cab25_5_ga_ts_tab_cap15.to_csv ('minimum_value_dataframe_cab25_5_ga_ts_tab_cap15.csv')